In [1]:
import pandas as pd
train=pd.read_csv('../input/medical-insurance-payout/expenses.csv')
train.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [2]:
!pip install flaml
from sklearn.metrics import mean_absolute_percentage_error

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.2/195.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.5/157.5 MB 7.3 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.6.1
    Uninstalling xgboost-1.6.1:
      Successfully uninstalled xgboost-1.6.1


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [4]:
from sklearn.model_selection import train_test_split
train, test = train_test_split( train, test_size=0.2, random_state=42,shuffle=True)
train.shape, test.shape

((1070, 7), (268, 7))

In [5]:
train[train.isna().any(axis=1)]

,age,sex,bmi,children,smoker,region,charges


In [6]:
test[test.isna().any(axis=1)]

,age,sex,bmi,children,smoker,region,charges


In [7]:
from flaml import AutoML
automl = AutoML()

In [8]:
y = train.pop('charges')
X = train

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [10]:
automl.fit(X_train, y_train, task="regression",metric='rmse',time_budget=900)

[flaml.automl: 06-06 15:58:35] {2373} INFO - task = regression
[flaml.automl: 06-06 15:58:35] {2375} INFO - Data split method: uniform
[flaml.automl: 06-06 15:58:35] {2379} INFO - Evaluation method: cv
[flaml.automl: 06-06 15:58:35] {2448} INFO - Minimizing error metric: rmse
[flaml.automl: 06-06 15:58:36] {2586} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl: 06-06 15:58:36] {2878} INFO - iteration 0, current learner lgbm
[flaml.automl: 06-06 15:58:36] {3009} INFO - Estimated sufficient time budget=2015s. Estimated necessary time budget=17s.
[flaml.automl: 06-06 15:58:36] {3061} INFO -  at 0.5s,	estimator lgbm's best error=8667.7514,	best estimator lgbm's best error=8667.7514
[flaml.automl: 06-06 15:58:36] {2878} INFO - iteration 1, current learner lgbm
[flaml.automl: 06-06 15:58:36] {3061} INFO -  at 0.6s,	estimator lgbm's best error=8667.7514,	best estimator lgbm's best error=8667.7514
[flaml.automl: 06-0

In [11]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best rmse on validation data: {0:.4g}'.format(automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: xgb_limitdepth
Best hyperparmeter config: {'n_estimators': 16, 'max_depth': 3, 'learning_rate': 0.346133397105234, 'subsample': 1.0, 'colsample_bylevel': 1.0, 'colsample_bytree': 1.0, 'reg_alpha': 0.010230251484079588, 'reg_lambda': 2.8271894172327214, 'min_child_weight': 0.001}
Best rmse on validation data: 4343
Training duration of best run: 0.02198 s


In [12]:
y_val=test.pop('charges')
X_val= test

In [13]:
from flaml.ml import sklearn_metric_loss_score
print('train rmse', '=', sklearn_metric_loss_score('rmse', automl.predict(X_train), y_train))
print(' test rmse', '=', sklearn_metric_loss_score('rmse', automl.predict(X_test), y_test))
print('  val rmse', '=', sklearn_metric_loss_score('rmse', automl.predict(X_val), y_val))

train rmse = 3865.1768393724465
 test rmse = 5276.71090852356
  val rmse = 4422.194413221822
